In [21]:
import os
import sys
import pandas as pd
sys.path.append('..')
from climate.txt_to_tables import txt_to_df, montly_columns_names_en

In [22]:
meteo_path = '../input/climate/meteo.ru/new'

In [24]:
station_ru2en = {
    'Хоседа-Хард': 'Khoseda Khard',
    'Тура': 'Tura',
    'Енисейск': 'Eniseysk',
    'Минусинск': 'Minusinsk',
    'Иркутск, обсерв': 'Irkutsk',
    'Мугур-Аксы': 'Mugur-Aksy',
    'Барнаул': 'Barnaul',
    'Якутск': 'Jakutsk',
    'Кандалакша': 'Kandalaksa',
    'Нижнеангарск': 'Nizneangarsk',
    'Эрзин': 'Ersin'
}

In [25]:
for char in os.listdir(meteo_path):
    char_path = f'{meteo_path}/{char}/'
    files = os.listdir(char_path)
    statlist_file = ''
    for file in files[::-1]:
        if 'statlist' in file:
            statlist_file = file
    wmo_codes = []
    station_names = []
    with open(f'{char_path}/{statlist_file}') as f:
        for line in f:
            wmo_code, *station_name = line.strip().split()
            station_name = ' '.join(station_name)
            wmo_codes.append(wmo_code)
            station_names.append(station_name)
    for wmo_code, station_name in zip(wmo_codes, station_names):
        df = txt_to_df(station=station_ru2en[station_name],
                       char=char,
                       colums_names=montly_columns_names_en,
                       input_path=f'{char_path}/{wmo_code}.txt',
                       output_path=char_path,
                       save_csv=True)

In [ ]:
path = '../input/climate/real'

characterisitict = [
    'precipitation',
    'relative humidity',
    'sunshine duration',
    'temperature',
    'vpd'
]

In [ ]:
for char in characterisitict:
    for file in os.listdir(f'{path}/{char}'):
        try:
            wmo_index, ch, loc_ru = file.split('_')
        except:
            continue
        loc_ru = loc_ru.split('.')[0]
        if loc_ru in station_ru2en:
            pd.read_csv(f'{path}/{char}/{file}').drop(columns=['WMO Index']).to_csv(f'{path}/{char}/{ch}_{station_ru2en[loc_ru]}.csv', index=False)

In [2]:
from climate.weather_gc_ca import *

In [ ]:
data = download_climate(49649, 2012, 2023)

In [28]:
dict_to_json(data, '../output/SCHEFFERVILLE A 1953-2010')
dict_to_csv(data, '../output/SCHEFFERVILLE A 1953-2010', 1953, 2011)

In [11]:
df = pd.read_csv('../output/SCHEFFERVILLE A_HOURLY 2012-2022.csv')

In [ ]:
daily_df = hourly_to_daily(df)
monthly_df = daily_to_monthly(daily_df)

In [13]:
daily_df.to_csv('../output/SCHEFFERVILLE A 2012-2022_daily.csv', index=False)
monthly_df.to_csv('../output/SCHEFFERVILLE A 2012-2022_monthly.csv', index=False)

In [54]:
df = pd.read_csv('../output/trash/SCHEFFERVILLE A 1953-2010_HOURLY.csv')

C:\Users\Mikhail\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10,12,14,16,18,20,22,24,26,27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [131]:
files = os.listdir('../output/trash')[:-2]

In [ ]:
columns = ['Year', 'Month', 'Day', 'Max Temp (°C)','Min Temp (°C)', 'Mean Temp (°C)', 'Total Precip (mm)']

agg_arg = { 
    'Year' : 'min', 'Max Temp (°C)': 'max','Min Temp (°C)': 'min', 'Mean Temp (°C)': 'mean',
    'Total Precip (mm)': 'sum'
}

In [139]:
Flag = False
final_df = None
for file in files:
    df = pd.read_csv(f'../output/trash/{file}')
    df = df[columns].groupby('Month').agg(agg_arg)
    if Flag:
        final_df = final_df.append(df)
    else:
        final_df = df
        Flag = True

In [142]:
final_df.reset_index().to_csv('../output/trash/Schefferville Monthly.csv', index=False)

In [152]:
final_df = final_df.reset_index()

In [156]:
pd.read_csv('../output/trash/Schefferville_1948-1993.csv')[['Year', 'Month', 'Mean Temp (°C)', 'Total Precip (mm)']].append(final_df[['Year', 'Month', 'Mean Temp (°C)', 'Total Precip (mm)']]).to_csv('../output/Schefferville Monthly FULL.csv', index=False)

In [158]:
df = pd.read_csv('../output/Schefferville Monthly FULL.csv')

In [160]:
df

,Year,Month,Mean Temp (°C),Total Precip (mm)
0,1948,1,NaN,NaN
1,1948,2,NaN,NaN
2,1948,3,NaN,NaN
3,1948,4,NaN,NaN
4,1948,5,NaN,NaN
...,...,...,...,...
847,2018,8,NaN,0.0
848,2018,9,NaN,0.0
849,2018,10,NaN,0.0
850,2018,11,NaN,0.0


In [163]:
montly_to_measurement(df, 'Mean Temp (°C)').to_csv('../output/Temp_Schefferville.csv', index=False)
montly_to_measurement(df, 'Total Precip (mm)').to_csv('../output/Prec_Schefferville.csv', index=False)


In [165]:
scand_df = pd.read_excel('../output/Isonet_Scandinavia_Data_Meteo.xlsx')

In [177]:
INA = scand_df[scand_df['Site']=='INA'].rename(columns={'YEAR':'Year', 'month':'Month'})
ILO = scand_df[scand_df['Site']=='ILO'].rename(columns={'YEAR':'Year', 'month':'Month'})
GUT = scand_df[scand_df['Site']=='GUT'].rename(columns={'YEAR':'Year', 'month':'Month'})

In [178]:
montly_to_measurement(INA, 'T').to_csv('../output/Temp_Kessi Inari.csv', index=False)
montly_to_measurement(INA, 'P').to_csv('../output/Prec_Kessi Inari.csv', index=False)
montly_to_measurement(INA, 'VPD').to_csv('../output/VPD_Kessi Inari.csv', index=False)

In [179]:
montly_to_measurement(ILO, 'T').to_csv('../output/Temp_Ilomantsi.csv', index=False)
montly_to_measurement(ILO, 'P').to_csv('../output/Prec_Ilomantsi.csv', index=False)
montly_to_measurement(ILO, 'VPD').to_csv('../output/VPD_Ilomantsi.csv', index=False)

In [180]:
montly_to_measurement(GUT, 'T').to_csv('../output/Temp_Gutulia.csv', index=False)
montly_to_measurement(GUT, 'P').to_csv('../output/Prec_Gutulia.csv', index=False)
montly_to_measurement(GUT, 'VPD').to_csv('../output/VPD_Gutulia.csv', index=False)

In [181]:
set(scand_df['Site'])

{'BRO',
 'CAV',
 'CAZ',
 'DRA',
 'GIB',
 'GUT',
 'ILO',
 'INA',
 'LCH',
 'LIL',
 'PAN',
 'PED',
 'POE',
 'SUW',
 'VIG',
 'WIN',
 'WOB'}